# 逻辑回归

## The data

我们将建立一个逻辑回归模型来预测一个学生是否被大学录取。假设你是一个大学里，系的管理员，你想根据两次考试的结果来决定每个申请人的录取机会。你有以前的申请人的历史数据，你可以用它作为逻辑回归的训练集。对于每一个培训例子，你有两个考试的申请人的分数和录取决定。为了做到这一点，我们将建立一个分类模型，根据考试成绩估计入学概率。

In [ ]:
#三大件
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import os
# 读取txt文件
path = 'data' + os.sep + 'LogiReg_data.txt'
# 用Pandas读取,并转换成DataFrame结构，设置列名：第一次考试成绩，第二次考试成绩，是否被录取
pdData = pd.read_csv(path, header=None, names=['Exam 1', 'Exam 2', 'Admitted'])
pdData.head()

In [ ]:
# 查看数据维度
pdData.shape 

In [ ]:
# 正例
positive = pdData[pdData['Admitted'] == 1]
# 负例
negative = pdData[pdData['Admitted'] == 0]
plt.style.use(['ggplot','bmh'])
# 指定画图区域大小
fig, ax = plt.subplots(figsize=(10,5))
# 正例散点图，设置：标记大小，颜色，标记样式，示例名
ax.scatter(positive['Exam 1'], positive['Exam 2'], s=30, c='b', marker='o', label='Admitted')
# 负例散点图，设置：标记大小，颜色，标记样式，示例名
ax.scatter(negative['Exam 1'], negative['Exam 2'], s=30, c='r', marker='x', label='Not Admitted')
# 显示示例
ax.legend()
# x,y轴的名称
ax.set_xlabel('Exam 1 Score')
ax.set_ylabel('Exam 2 Score')

## The logistic regression

目标：建立分类器（求解出三个参数 $\theta_0         \theta_1         \theta_2 $）


设定阈值，根据阈值判断录取结果

### 要完成的模块
-  `sigmoid` : 由值到概率的映射函数

-  `model` : 返回预测结果值

-  `cost` : 根据参数计算损失

-  `gradient` : 计算每个参数的梯度方向

-  `descent` : 进行参数更新

-  `accuracy`: 计算精度

###  `sigmoid` 函数

$$
g(z) = \frac{1}{1+e^{-z}}
$$

In [ ]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [ ]:
# 看一下sigmoid是什么样子
nums = np.arange(-10, 10, step=1)
fig, ax = plt.subplots(figsize=(12,4))
ax.plot(nums, sigmoid(nums), 'r')

### Sigmoid 取值范围
* $g:\mathbb{R} \to [0,1]$
* $g(0)=0.5$
* $g(- \infty)=0$
* $g(+ \infty)=1$

In [ ]:
# model是预测函数，X数据，theta参数
def model(X, theta):
    # 做一个矩阵乘法
    return sigmoid(np.dot(X, theta.T))

$$
\begin{array}{ccc}
\begin{pmatrix}\theta_{0} & \theta_{1} & \theta_{2}\end{pmatrix} & \times & \begin{pmatrix}1\\
x_{1}\\
x_{2}
\end{pmatrix}\end{array}=\theta_{0}+\theta_{1}x_{1}+\theta_{2}x_{2}
$$

In [ ]:
# 插入一列，全为1的值
pdData.insert(0, 'Ones', 1)
orig_data = pdData.values
cols = orig_data.shape[1]
X = orig_data[:,0:cols-1]
y = orig_data[:,cols-1:cols]
#X = np.matrix(X.values)
#y = np.matrix(data.iloc[:,3:4].values) #np.array(y.values)
# 用0来填充并构造一个1行3列的矩阵，用来占位
theta = np.zeros([1, 3])

随做，随时检查

In [ ]:
# 看一下X
X[:5]

In [ ]:
# 看一下y
y[:5]

In [ ]:
# 看一下theta
theta

In [ ]:
# 查看一下3个变量的维度
X.shape, y.shape, theta.shape

### 损失函数
将对数似然函数去负号

$$
D(h_\theta(x), y) = -y\log(h_\theta(x)) - (1-y)\log(1-h_\theta(x))
$$
求平均损失
$$
J(\theta)=\frac{1}{n}\sum_{i=1}^{n} D(h_\theta(x_i), y_i)
$$

In [ ]:
# 损失函数，X样本数据，y是标签，theta参数
def cost(X, y, theta):
    left = np.multiply(-y, np.log(model(X, theta)))
    right = np.multiply(1 - y, np.log(1 - model(X, theta)))
    return np.sum(left - right) / (len(X))

In [ ]:
cost(X, y, theta)

### 计算梯度

偏导

$$
\frac{\partial J}{\partial \theta_j}=-\frac{1}{m}\sum_{i=1}^n (y_i - h_\theta (x_i))x_{ij}
$$


In [ ]:
# 计算梯度函数
def gradient(X, y, theta):
    # 声明梯度的变量
    grad = np.zeros(theta.shape)
    error = (model(X, theta)- y).ravel()
    # 每次的偏导，要求3此theta的偏导
    for j in range(len(theta.ravel())):
        term = np.multiply(error, X[:,j])
        # 计算梯度
        grad[0, j] = np.sum(term) / len(X)
    
    return grad

### Gradient descent

比较3中不同梯度下降方法


In [ ]:
# 按照迭代次数停止
STOP_ITER = 0
# 根据损失停止
STOP_COST = 1
# 根据梯度停止
STOP_GRAD = 2

def stopCriterion(type, value, threshold):
    # 设定三种不同的停止策略
    if type == STOP_ITER:        return value > threshold # 指定一个次数
    elif type == STOP_COST:      return abs(value[-1]-value[-2]) < threshold # 指定阈值
    elif type == STOP_GRAD:      return np.linalg.norm(value) < threshold # 指定阈值

In [ ]:
import numpy.random
# 洗牌，打乱顺序，让有规律的数据无规律，增强函数得泛化能力
def shuffleData(data):
    # 使用random中的shuffle
    np.random.shuffle(data)
    cols = data.shape[1]
    X = data[:, 0:cols-1]
    y = data[:, cols-1:]
    return X, y

In [ ]:
import time
# 不同的梯度下降策略，消耗的时间不同，要看一下，时间对结果的影响
# data数据，theta参数，batchSize梯度策略，stopType停止策略，thresh策略对应的阈值，alpha学习率
def descent(data, theta, batchSize, stopType, thresh, alpha):
    #梯度下降求解，先做初始化
    init_time = time.time()
    i = 0 # 迭代次数
    k = 0 # batch
    X, y = shuffleData(data)
    grad = np.zeros(theta.shape) # 计算的梯度
    costs = [cost(X, y, theta)] # 损失值

    while True:
        # 根据不同策略，使用实际的x,y，进行梯度计算
        grad = gradient(X[k:k+batchSize], y[k:k+batchSize], theta)
        k += batchSize #取batch数量个数据
        if k >= n: 
            k = 0 
            X, y = shuffleData(data) #重新洗牌
        theta = theta - alpha*grad # 参数更新
        costs.append(cost(X, y, theta)) # 计算新的损失
        i += 1
        # 判断，是否停止
        if stopType == STOP_ITER:       value = i
        elif stopType == STOP_COST:     value = costs
        elif stopType == STOP_GRAD:     value = grad
        # break，进行停止
        if stopCriterion(stopType, value, thresh): break
    
    return theta, i-1, costs, grad, time.time() - init_time

In [ ]:
def runExpe(data, theta, batchSize, stopType, thresh, alpha):
    #import pdb; pdb.set_trace();
    # 初始化，求解
    theta, iter, costs, grad, dur = descent(data, theta, batchSize, stopType, thresh, alpha)
    # 定义如何显示信息，根据参数，选择梯度下降方式以及停止策略
    name = "Original" if (data[:,1]>2).sum() > 1 else "Scaled"
    name += " data - learning rate: {} - ".format(alpha)
    if batchSize==n: strDescType = "Gradient"
    elif batchSize==1:  strDescType = "Stochastic"
    else: strDescType = "Mini-batch ({})".format(batchSize)
    name += strDescType + " descent - Stop: "
    # 停止
    if stopType == STOP_ITER: strStop = "{} iterations".format(thresh)
    elif stopType == STOP_COST: strStop = "costs change < {}".format(thresh)
    else: strStop = "gradient norm < {}".format(thresh)
    name += strStop
    # 展示与画图
    print ("***{}\nTheta: {} - Iter: {} - Last cost: {:03.2f} - Duration: {:03.2f}s".format(
        name, theta, iter, costs[-1], dur))
    plt.style.use(['ggplot','bmh'])
    fig, ax = plt.subplots(figsize=(12,4))
    ax.plot(np.arange(len(costs)), costs, 'r')
    ax.set_xlabel('Iterations')
    ax.set_ylabel('Cost')
    ax.set_title(name.upper() + ' - Error vs. Iteration')
    return theta

### 不同的停止策略

#### 设定迭代次数

In [ ]:
# 选择的梯度下降方法是基于所有样本的
n=100
# 按照迭代次数STOP_ITER，迭代5000次thresh=5000，学习率alpha=0.000001
runExpe(orig_data, theta, n, STOP_ITER, thresh=5000, alpha=0.000001)

#### 根据损失值停止

In [ ]:
# 损失函数STOP_COST，两次之间小于thresh，
runExpe(orig_data, theta, n, STOP_COST, thresh=0.000001, alpha=0.001)

#### 根据梯度变化停止

In [ ]:
# 梯度STOP_GRAD，阈值小于thresh=0.05
runExpe(orig_data, theta, n, STOP_GRAD, thresh=0.05, alpha=0.001)

### 对比不同的梯度下降方法

#### Stochastic descent

In [ ]:
# 1个样本
runExpe(orig_data, theta, 1, STOP_ITER, thresh=5000, alpha=0.001)

有点爆炸。。。很不稳定,再来试试把学习率调小一些

In [ ]:
runExpe(orig_data, theta, 1, STOP_ITER, thresh=15000, alpha=0.000002)

速度快，但稳定性差，需要很小的学习率

#### Mini-batch descent

In [ ]:
# 样本数16
runExpe(orig_data, theta, 16, STOP_ITER, thresh=15000, alpha=0.001)

浮动仍然比较大，我们来尝试下对数据进行标准化
将数据按其属性(按列进行)减去其均值，然后除以其方差。最后得到的结果是，对每个属性/每列来说所有数据都聚集在0附近，方差值为1

In [ ]:
from sklearn import preprocessing as pp
# 处理后的数据scaled_data
scaled_data = orig_data.copy()
scaled_data[:, 1:3] = pp.scale(orig_data[:, 1:3])

runExpe(scaled_data, theta, n, STOP_ITER, thresh=5000, alpha=0.001)

它好多了！原始数据，只能达到达到0.63左右，而我们得到了0.38个在这里！
所以对数据做预处理是非常重要的
### 有问题，先从数据层面下手

In [ ]:
# 样本100，使用梯度，阈值调整到thresh=0.02
runExpe(scaled_data, theta, n, STOP_GRAD, thresh=0.02, alpha=0.001)

更多的迭代次数会使得损失下降的更多！

In [ ]:
theta = runExpe(scaled_data, theta, 1, STOP_GRAD, thresh=0.002/5, alpha=0.001)

随机梯度下降更快，但是我们需要迭代的次数也需要更多，所以还是用batch的比较合适！！！

In [ ]:
runExpe(scaled_data, theta, 16, STOP_GRAD, thresh=0.002*2, alpha=0.001)

## 精度

In [ ]:
#设定阈值
def predict(X, theta):
    # 大于0.5被录取，小于0.5不被录取
    return [1 if x >= 0.5 else 0 for x in model(X, theta)]

In [ ]:
scaled_X = scaled_data[:, :3]
y = scaled_data[:, 3]
predictions = predict(scaled_X, theta)
# 精度，预测对了多少个
correct = [1 if ((a == 1 and b == 1) or (a == 0 and b == 0)) else 0 for (a, b) in zip(predictions, y)]
accuracy = (sum(map(int, correct)) % len(correct))
print ('accuracy = {0}%'.format(accuracy))